# Connect to Drive

In [1]:
# connect with drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')
my_year_month_fmt = mdates.DateFormatter('%m/%y')
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# install library sastrawi
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
# import library
import nltk
import pandas as pd
import re
import time
import multiprocessing
import io
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.utils.multiclass import unique_labels

# Route to Directory Dataset

In [ ]:
# save path to dataset
path_data_sentimen = "/content/drive/MyDrive/Semester 5/Pemrograman Sistem Cerdas/Prak_Riky raharjo/Praktikum3/halodoc - halodoc (3).csv"
data_sentiment = pd.read_csv(path_data_sentimen, sep=",", header=[0], encoding="UTF-8")

In [ ]:
data_sentiment.head()

,content,score
0,Sudah update versi terbaru tetap saja utk sett...,0
1,Untuk toko kesehatan kurirnya tolong diperbaik...,0
2,"Saat buka aplikasi loading terus, sampe akhirn...",0
3,Bikin hp error. Gak bisa kembali dan klo beli ...,0
4,Kalo ngga niat promo mending ngga usah adain p...,0


In [ ]:
unique_labels(data_sentiment['score'])
#0 Sebagai Negatif
#1 sebagai Positif
#-1 Sebagai Netral

array([-1,  0,  1])

In [ ]:
#menyimpan content. (tipe data series pandas)
data_content = data_sentiment['content']

#Preprocessing

##Case Folding

In [ ]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    sudah update versi terbaru tetap saja utk sett...
1    untuk toko kesehatan kurirnya tolong diperbaik...
2    saat buka aplikasi loading terus, sampe akhirn...
3    bikin hp error. gak bisa kembali dan klo beli ...
4    kalo ngga niat promo mending ngga usah adain p...
Name: content, dtype: object

##Filtering

In [ ]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

##Tokenization

In [ ]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(content) for content in data_filtering]

##Save clear dataset

In [ ]:
# Menyimpan hasil preprocessing ke dalam DataFrame
df_preprocessed = pd.DataFrame({'Original_Content': data_content, 'Preprocessed_Content': data_filtering})

# Menyimpan DataFrame ke dalam file CSV
path_output_csv = "/content/drive/MyDrive/Semester 5/Capstone Project/Dataset/Sentiment Analysis/kamus_bahasa_baku_halodoc1000.csv"
df_preprocessed.to_csv(path_output_csv, index=False)

print(f"Hasil preprocessing disimpan dalam file CSV: {path_output_csv}")


Hasil preprocessing disimpan dalam file CSV: /content/drive/MyDrive/Semester 5/Capstone Project/Dataset/Sentiment Analysis/kamus_bahasa_baku_halodoc1000.csv


##Slang word

In [ ]:
#slang word
path_dataslang = open("/content/drive/MyDrive/Semester 5/Capstone Project/Dataset/Sentiment Analysis/kamus_bahasa_baku_halodoc1000.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=",")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
print(data_formal)
len_data_formal

[['sudah', 'update', 'versi', 'terbaru', 'tetap', 'saja', 'utk', 'setting', 'alamat', 'via', 'maps', 'tetap', 'ga', 'sesuai', 'titik', 'sudah', 'pas', 'tp', 'alamatnya', 'beda', 'pdhl', 'di', 'aplikasi', 'lain', 'utk', 'setting', 'alamat', 'ga', 'ada', 'masalah', 'cuma', 'halodoc', 'aja', 'yg', 'ribet', 'jd', 'males', 'pake', 'aplikasi', 'ini'], ['untuk', 'toko', 'kesehatan', 'kurirnya', 'tolong', 'diperbaiki', 'lagi', 'udah', 'pilih', 'yang', 'instant', 'jam', 'gak', 'dateng', 'ga', 'ada', 'bedanya', 'sama', 'instant', 'hemat', 'jam', 'dan', 'sisten', 'pembatalan', 'pesanan', 'tolong', 'diperbaiki', 'saya', 'jadi', 'beli', 'obat', 'gara', 'terlalu', 'lama', 'datangnya', 'tolong', 'next', 'diperbaiki', 'kurirnya', 'karena', 'ini', 'untuk', 'orang', 'yang', 'sakit', 'setidaknya', 'tepat', 'waktu', 'dan', 'kurirnya', 'punya', 'nomor', 'aktif', 'yang', 'bisa', 'ditelepon', 'sms'], ['saat', 'buka', 'aplikasi', 'loading', 'terus', 'sampe', 'akhirnya', 'coba', 'reinstall', 'setelah', 'login'

3000

##Stopword Removal

In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Stemming

In [ ]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
print(reviews)

[['update', 'versi', 'baru', 'setting', 'alamat', 'maps', 'sesuai', 'titik', 'alamat', 'beda', 'pdhl', 'aplikasi', 'setting', 'alamat', 'halodoc', 'ribet', 'males', 'pake', 'aplikasi'], ['toko', 'sehat', 'kurir', 'tolong', 'baik', 'udah', 'pilih', 'instant', 'dateng', 'beda', 'instant', 'hemat', 'sisten', 'batal', 'pesan', 'tolong', 'baik', 'beli', 'obat', 'gara', 'datang', 'tolong', 'next', 'baik', 'kurir', 'orang', 'sakit', 'kurir', 'nomor', 'aktif', 'telepon'], ['buka', 'aplikasi', 'loading', 'sampe', 'coba', 'reinstall', 'login', 'halaman', 'dashboard', 'tetep', 'loading', 'tombol', 'pencet', 'gabisa', 'developer', 'gimana', 'baik', 'nyaman', 'guna'], ['bikin', 'error', 'beli', 'obat', 'aplikasi', 'langsung', 'error', 'menu', 'utama', 'restart', 'tunggu', 'loading', 'halodoc', 'uninstall', 'bikin', 'emosi'], ['kalo', 'ngga', 'niat', 'promo', 'mending', 'ngga', 'adain', 'promo', 'pakai', 'kode', 'promo', 'pesan', 'langsung', 'cancel', 'refund', 'aplikasi', 'ngga', 'login', 'parah'],

##Tf-IDF

In [ ]:
# pembuatan vector kata
# reviews = pickle.load(open("/content/drive/My Drive/Thesis/Tesis/program/data/model/NVB/reviews.pickle", "rb"))
# pembuatan vector kata
vectorizer = TfidfVectorizer()
reviews2 = [" ".join(r) for r in reviews]
vektor_tfidf = vectorizer.fit_transform(reviews2)
vektor_tfidf = vektor_tfidf.toarray()
vektor_tfidf.shape

(3000, 4158)

In [ ]:
reviews2

['update versi baru setting alamat maps sesuai titik alamat beda pdhl aplikasi setting alamat halodoc ribet males pake aplikasi',
 'toko sehat kurir tolong baik udah pilih instant dateng beda instant hemat sisten batal pesan tolong baik beli obat gara datang tolong next baik kurir orang sakit kurir nomor aktif telepon',
 'buka aplikasi loading sampe coba reinstall login halaman dashboard tetep loading tombol pencet gabisa developer gimana baik nyaman guna',
 'bikin error beli obat aplikasi langsung error menu utama restart tunggu loading halodoc uninstall bikin emosi',
 'kalo ngga niat promo mending ngga adain promo pakai kode promo pesan langsung cancel refund aplikasi ngga login parah',
 'simpan alamat manual alamat tentu otomatis lokasi sesuai alamat muncul beda masukin alamat manual alamat rekomendasi baik guna fitur simpan alamat disetting alamat manual contoh fitur simpan alamat commerce simpan alamat manual akurat sesuai alamat alamat ribet',
 'tolong daftar list obat tambah bah

In [ ]:
import pickle

# write list to binary file
def write_list(a_list):
    # store list in binary file so 'wb' mode
    with open('/content/drive/MyDrive/Semester 5/Capstone Project/Dataset/Sentiment Analysis/training.pickle', 'wb') as fp:
        pickle.dump(names, fp)
        print('Done writing list into a binary file')
# list of names
names = reviews2
write_list(names)

Done writing list into a binary file


In [ ]:
vektor_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#Panggil Data target aspek (y)
score = data_sentiment['score']
score.shape

(3000,)

#Split Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vektor_tfidf, score, test_size=0.2)
nama = "Support Vektor Machine"
y_train = np.array(y_train)
y_test = np.array(y_test)

#Training Model

In [ ]:
from sklearn.metrics import classification_report
import joblib
# ...

# Membuat model SVM
svm_model = SVC(kernel='rbf')
'''
svm_model = SVC(kernel='linear')
'''

# Melatih model pada data latih
svm_model.fit(X_train, y_train)

# Memprediksi label untuk data uji
y_pred_svm = svm_model.predict(X_test)

# Evaluasi model SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'Akurasi SVM: {accuracy_svm}')

# Simpan model SVM ke dalam file
path_model_svm = "/content/drive/MyDrive/Semester 5/Capstone Project/Dataset/Sentiment Analysis/model_svm.pkl"
joblib.dump(svm_model, path_model_svm)

print(f"Model SVM telah disimpan di: {path_model_svm}")

# ...

# Load model SVM dari file
model_svm_loaded = joblib.load(path_model_svm)

# Memprediksi label untuk data uji menggunakan model yang telah di-load
pred_y_svm = model_svm_loaded.predict(X_test)

# Evaluasi model SVM setelah di-load
accuracy_svm_loaded = accuracy_score(y_test, pred_y_svm)
print(f'Akurasi SVM setelah di-load: {accuracy_svm_loaded}')

# Menampilkan confusion matrix
print('\n' + '-' * 45)
print('Confusion matrix SVM setelah di-load:')
print('-' * 45, '\n')
print(classification_report(y_test, pred_y_svm, labels=[-1, 0, 1]))


Akurasi SVM: 0.695
Model SVM telah disimpan di: /content/drive/MyDrive/Semester 5/Capstone Project/Dataset/Sentiment Analysis/model_svm.pkl
Akurasi SVM setelah di-load: 0.695

---------------------------------------------
Confusion matrix SVM setelah di-load:
--------------------------------------------- 

              precision    recall  f1-score   support

          -1       0.58      0.56      0.57       196
           0       0.70      0.76      0.73       195
           1       0.80      0.76      0.78       209

    accuracy                           0.69       600
   macro avg       0.69      0.69      0.69       600
weighted avg       0.70      0.69      0.69       600



In [ ]:
new_text = input("\n Masukkan komentar : ")
new_text_vec = vectorizer.transform([new_text])

# Konversi vektor sparse menjadi dense
new_text_vec_dense = new_text_vec.toarray()

predicted_sentimen = model_svm_loaded.predict(new_text_vec_dense)

if predicted_sentimen[0] == 1:
    sentiment_label = "positif"
elif predicted_sentimen[0] == 0:
    sentiment_label = "netral"
elif predicted_sentimen[0] == -1:
    sentiment_label = "negatif"

print("Hasil analisis sentimen adalah:", sentiment_label)


 Masukkan komentar : bagus banget sih
Hasil analisis sentimen adalah: positif
